Mini Transformer - BUMBLEBEE - CEL: zbudować mały model jezykowy na self-attension z jedną warstwą transformera, która potrafi uczyć się prostych zależnosci w sekwencjach (przwidywanie następnego tokenu)

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

KROK2: definiujemy poejdynczy blok transformera

In [3]:
class TransformerBlock(nn.Module):
    def __init__(self, embed_dim):
        super(TransformerBlock, self).__init__()
        self.key = nn.Linear(embed_dim, embed_dim,bias=False)
        self.query = nn.Linear(embed_dim, embed_dim,bias=False)
        self.value = nn.Linear(embed_dim, embed_dim,bias=False)
        self.proj = nn.Linear(embed_dim, embed_dim)
        self.ln = nn.LayerNorm(embed_dim)

        #KROK3: mechanizm self-attention

        def forward(self, x):
            B, T, C = x.shape #B=batch, T=sequence length, C=embedding size
            k = self.key(x)
            q = self.query(x)
            v = self.value(x)

            scores = q @ k.transpose(-2, -1) / (C**0.5) #B,T,T -> oblicza podobieństwo między tokenami!
            mask = torch.tril(torch.ones(T,T)).to(x.device) #nie patrzymy na przyszłe tokeny - (ważne w modelach generatywnych)
            scores = scores.masked_fill(mask == 0,float('-inf')) #maskowanie przyszłości

            att = F.softmax(scores, dim=-1) #rozkład uwagi
            out=att@v #mieszanie wartości zgodnie z uwagą , v to "treść" tokenu który bierzemy
            return self.ln(self.proj(out) + x) #resztkowe połączenie i normalizacja

